<a href="https://colab.research.google.com/github/aaalexlit/omdena_climate_change_challenge_notebooks/blob/main/Verify_claims_with_multivers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multivers Setup

## Clone multivers repo

In [ ]:
%%capture
!git clone https://github.com/dwadden/multivers.git

## Setup virtual environment and requirements
virtual environment is needed to avoid conflicts with the packages already installed on Colab

In [ ]:
%%capture
!pip install virtualenv
!virtualenv multivers
!source /content/multivers/bin/activate; pip install -r /content/multivers/requirements.txt

## Download the model checkpoints

In [ ]:
!python /content/multivers/script/get_checkpoint.py longformer_large_science
!python /content/multivers/script/get_checkpoint.py scifact


--2023-03-05 13:33:42--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/longformer_large_science.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.218.234.169, 3.5.76.175, 52.92.128.34, ...
Connecting to scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)|52.218.234.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1646843693 (1.5G) [binary/octet-stream]
Saving to: ‘checkpoints/longformer_large_science.ckpt’

checkpoints/longfor 100%[===================>]   1.53G  7.67MB/s    in 98s     

2023-03-05 13:35:21 (16.0 MB/s) - ‘checkpoints/longformer_large_science.ckpt’ saved [1646843693/1646843693]

--2023-03-05 13:35:21--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/scifact.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 3.5.80.155, 52.92.152.10, 3.5.77.132, ...
Connecting to scifact.s3.us-west-2.amazo

And move the models to the folder where multivers is looking for them

In [ ]:
!mv /content/checkpoints /content/multivers/checkpoints

## If CPU is not available 
delete line `    model.to(f"cuda:{args.device}")` from `predict.py` otherwise it would throw an error `RuntimeError: No CUDA GPUs are available
`

In [ ]:
import torch
import os
if not torch.cuda.is_available():
  ret_code = os.system("sed -i '40d' /content/multivers/multivers/predict.py")
  print(f"exit code = {ret_code}")

# Setup evidence retrieval

## Download abstracts database

In [ ]:
## we need this to be able to download big-ish files from GDrive
%%capture
!pip3 install --upgrade gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1b_FWp-526YsVOvonkEj_qvtHZFZusN8y -O faiss_document_store.db
!gdown https://drive.google.com/uc?id=1--xpfesTzHcwrGw4NsZI4lFaQWnzHuZz -O faiss_index
!gdown https://drive.google.com/uc?id=1-2pdEl2JBs1KUpLRCoiDhVSE1yGkHoGI -O faiss_index.json

Downloading...
From: https://drive.google.com/uc?id=1b_FWp-526YsVOvonkEj_qvtHZFZusN8y
To: /content/faiss_document_store.db
100% 1.06G/1.06G [00:05<00:00, 197MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--xpfesTzHcwrGw4NsZI4lFaQWnzHuZz
To: /content/faiss_index
100% 879M/879M [00:03<00:00, 227MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-2pdEl2JBs1KUpLRCoiDhVSE1yGkHoGI
To: /content/faiss_index.json
100% 187/187 [00:00<00:00, 377kB/s]


## Install requirements

### Check if GPU is available to install gpu version of faiss

In [ ]:
faiss_to_install = "faiss-gpu"
if not torch.cuda.is_available():
  faiss_to_install = "faiss"

ret_code = os.system(f"source /content/multivers/bin/activate; pip install farm-haystack[{faiss_to_install}]")
if not ret_code:
  print(f"Installed {faiss_to_install}")

Installed faiss-gpu


In [ ]:
!pip install newspaper3k
!pip install codetiming
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [ ]:
from newspaper import Article
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
                         pipeline, RobertaForSequenceClassification
from codetiming import Timer
import logging

from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever  
import jsonlines

from timeit import default_timer as timer
from collections import defaultdict

import os
import torch 

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Setups and model downloads

In [ ]:
# model to build semantic encodings
MODEL_NAME = 'sebastian-hofstaetter/distilbert-dot-tas_b-b256-msmarco'
# embedding size used by distilbert-dot-tas_b-b256-msmarco
EMBEDDING_DIM = 768

SIMILARITY = 'dot_product'

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)

In [ ]:
claimbuster_tokenizer = AutoTokenizer.from_pretrained("lucafrost/ClaimBuster-DeBERTaV2")
claimbuster_model = AutoModelForSequenceClassification.from_pretrained("lucafrost/ClaimBuster-DeBERTaV2")

# climatebert_tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")
# climate_attention_model = RobertaForSequenceClassification.from_pretrained('kruthof/climateattention-ctw',
#                                                                            num_labels=2)

In [ ]:
class FAISSIndexer():
    def __init__(self, path_to_index_dir,
                 model_name, embedding_dim,
                 similarity_measure,
                 path_to_postgres=None) -> None:
        self.path_to_index_dir = path_to_index_dir
        # our db is postgres, only need to set path to faiss index
        if path_to_postgres:
            self.path_to_db = path_to_postgres
            self._set_path_to_index()
        # our db is SQLLite
        else:
            self._set_path_to_index_and_db()
        self.embedding_dim = embedding_dim
        self.model_name = model_name
        self.similarity_measure = similarity_measure
        self.document_store = self._init_document_store()
        self.retriever = self._init_retriever()

    def _set_path_to_index(self):
        if not os.path.exists(self.path_to_index_dir):
            os.makedirs(self.path_to_index_dir)
        self.path_to_index = os.path.join(self.path_to_index_dir, "faiss_index")

    def _set_path_to_index_and_db(self):
        self._set_path_to_index()
        self.path_to_db = f"sqlite:///{os.path.join(self.path_to_index_dir, 'faiss_document_store.db')}"

    def _init_document_store(self):
        if os.path.exists(self.path_to_index):
            return FAISSDocumentStore.load(index_path=self.path_to_index)
        else:
            return FAISSDocumentStore(
                sql_url=self.path_to_db,
                return_embedding=True,
                similarity=self.similarity_measure,
                embedding_dim=self.embedding_dim,
                duplicate_documents='skip'
            )

    def _init_retriever(self, progress_bar=True):
        return EmbeddingRetriever(
            document_store=self.document_store,
            embedding_model=self.model_name,
            model_format='sentence_transformers',
            # include article title into the embedding
            embed_meta_fields=["title"],
            progress_bar=progress_bar
        )

    def write_documents(self, docs):
        self.document_store.write_documents(docs)

        print('Updating embeddings ...')

        self.document_store.update_embeddings(
            retriever=self.retriever,
            update_existing_embeddings=False
        )

        print(f'current embedding count is {self.document_store.get_embedding_count()}')
        self.document_store.save(index_path=self.path_to_index)

    def retrieve_matches_for_a_phrase(self, phrase, top_k=10):
        return self.retriever.retrieve(phrase, top_k=top_k)

    def retrieve_matches_for_phrases(self, phrases, top_k=10):
        return self.retriever.retrieve_batch(phrases, top_k=top_k)

# Retrieve relevant evidences and convert them to Multivers format

In [ ]:
def is_claim(sentences,
             model=claimbuster_model,
             tokenizer=claimbuster_tokenizer,
             debug=False):
    if torch.cuda.is_available():
        device = 0
        batch_size = 128
    else:
        device = -1
        batch_size = 1
    pipe = pipeline("text-classification", model=model,
                    tokenizer=tokenizer, device=device)
    labels, probs = [], []
    for out in pipe(sentences, batch_size=batch_size):
        labels.append(out['label'])
        probs.append(out['score'])
    if debug:
        for sentence, label, prob in zip(sentences, labels, probs):
            print(f"{label}({prob:.3f})")
            print(sentence)
    return list(map(lambda l: model.config.label2id[l], labels)), probs

In [ ]:
def get_claims_from_text(text, threshold=0.5, debug=False):
    sentences = sent_tokenize(text)
    predicted_class_ids, probs = is_claim(sentences, debug=debug)
    return [sentence for sentence, label, prob in zip(sentences, predicted_class_ids, probs) if
            label in [1, 2] and prob > threshold]

In [ ]:
def get_text_from_url(news_article_url):
    article = Article(news_article_url)
    article.download()
    article.parse()
    article.nlp()
    return article.text

In [ ]:
@Timer(text="get_abstracts_matching_claims elapsed time: {seconds:.0f} s")
def get_abstracts_matching_claims(claims,
                                  indexer,
                                  top_k=10, debug=False):
    all_matches = indexer.retrieve_matches_for_phrases(claims,
                                                       top_k=top_k)
    if debug:
        for claim_n, matches in enumerate(all_matches):
            print(f"Claim:\n{claims[claim_n]}\n")
            for i, match in enumerate(matches):
                print(f'Evidence {i}:\n',
                      f'Similarity: {match.score:.3f}\n'
                      f'Quote: {match.content}\n',
                      f'Article Title: {match.meta.get("title", "")}\n',
                      f'DOI: {match.meta.get("doi", "")}\n',
                      f'year: {match.meta.get("publication_year", "")}\n', )
    return all_matches

In [ ]:
# https://github.com/dwadden/multivers/blob/main/doc/data.md
def convert_evidences_from_abstracts_to_multivers_format(text, indexer,
                                                         claims_filename,
                                                         corpus_filename):
    if text.startswith('http'):
        text = get_text_from_url(text)
    claims = get_claims_from_text(text, threshold=0.5)
    evidence_abstracts = get_abstracts_matching_claims(claims, indexer, top_k=30)
    with jsonlines.open(claims_filename, 'w') as claims_writer, \
            jsonlines.open(corpus_filename, 'w') as corpus_writer:
        doc_id = 0

        for claim_id, matches in enumerate(evidence_abstracts):
            doc_ids = []
            for i, match in enumerate(matches):
                evidence_abstract = {
                    'doc_id': doc_id,
                    'title': match.meta.get("title", ""),
                    'abstract': sent_tokenize(match.content)
                }
                corpus_writer.write(evidence_abstract)
                doc_ids.append(doc_id)
                doc_id += 1
            claim_doc = {
                'id': claim_id,
                'claim': claims[claim_id],
                'doc_ids': doc_ids
            }
            claims_writer.write(claim_doc)

In [ ]:
CLAIMS_FILE = "claims.jsonl"
CORPUS_FILE = "corpus.jsonl"
PREDS_FILE = "preds.jsonl"

In [ ]:
STORE_PATH = '/content/'
faiss_indexer = FAISSIndexer(STORE_PATH, 
                             MODEL_NAME, 
                             EMBEDDING_DIM,
                             # this needs to be changed to SIMILARITY which is dot_product when I re-index the abstracts correctly cause the used model was optimized for dot_product
                             'cosine')
# get_abstracts_matching_claims(
#     ["Rain now falls in massive bursts more frequently than it once did, triggering record floods.",
#       "CO2 is not the cause of our current warming trend."],
#     faiss_indexer,
#     debug=True);


# convert_evidences_from_abstracts_to_multivers_format("https://www.nationalgeographic.com/environment/article/amazon-rainforest-now-appears-to-be-contributing-to-climate-change",
#                                                      indexer=faiss_indexer,
#                                                      claims_filename=CLAIMS_FILE,
#                                                      corpus_filename=CORPUS_FILE)

convert_evidences_from_abstracts_to_multivers_format("https://www.thesun.co.uk/news/15288148/climate-change-risks-economy/",
                                                     indexer=faiss_indexer,
                                                     claims_filename=CLAIMS_FILE,
                                                     corpus_filename=CORPUS_FILE)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

get_abstracts_matching_claims elapsed time: 7 s


# First inference attempt

In [ ]:
start = timer()

In [ ]:
!source /content/multivers/bin/activate; python /content/multivers/multivers/predict.py \
        --checkpoint_path=/content/multivers/checkpoints/scifact.ckpt \
        --batch_size=1 \
        --input_file="/content/claims.jsonl" \
        --corpus_file="/content/corpus.jsonl" \
        --output_file="/content/preds.jsonl"

100% 420/420 [01:33<00:00,  4.48it/s]


In [ ]:
import gc
gc.collect()

5

In [ ]:
end = timer()
print(f"Took {(end - start):.0f} seconds")

Took 154 seconds


In [ ]:
# !cat /content/preds.jsonl

In [ ]:
def get_verified_claims(preds_filename, claims_filename, corpus_filename):
  claim_id_evidence_ids = {}
  with jsonlines.open(preds_filename, 'r') as preds_reader:
    for jsonline in preds_reader.iter():
      if jsonline["evidence"]:
        evidences = []
        for key in jsonline["evidence"]:
          d = {'evidence_id': key, 
              'label': jsonline["evidence"][key]['label'],
              'sentences': jsonline["evidence"][key]['sentences']
              }
          evidences.append(d)
        claim_id_evidence_ids[jsonline["id"]] = {"evidences": evidences}

  for claim_id in claim_id_evidence_ids:
    with jsonlines.open(claims_filename, 'r') as claims_reader:
      for line_num, jsonline in enumerate(claims_reader.iter()):
        if line_num == claim_id:
          claim_id_evidence_ids[claim_id]['claim_text']=jsonline['claim']

  for claim_id in claim_id_evidence_ids:
    evidences = claim_id_evidence_ids[claim_id]['evidences']
    for evidence in evidences:
      with jsonlines.open(corpus_filename, 'r') as corpus_reader:
        for line_num, jsonline in enumerate(corpus_reader.iter()):
          if str(line_num) == evidence['evidence_id']:
            evidence['evidence_title'] = jsonline['title']
            evidence['evidence_text'] = jsonline['abstract']
            sentences_text = []
            for sent_num in evidence['sentences']:
              sentences_text.append(jsonline['abstract'][sent_num])
            evidence['sentences_text'] = sentences_text
    return claim_id_evidence_ids

In [ ]:
verified_claims = get_verified_claims()

In [ ]:
for claim_id, claim in verified_claims.items():
  print('\nClaim:\n\t', claim['claim_text'])
  for evidence in claim['evidences']:
    print(f"\nLabel: {evidence['label']}")
    print(f"Article titile: {evidence['evidence_title']}")
    if evidence['sentences']:
      for sent in evidence['sentences_text']:
        print(f"Phrase: {sent}")
    else:
      print(f"Abstract: {evidence['evidence_text']}")
    


Claim:
	 But longer, hotter summers will lead to a tripling of heat-related deaths to around 7,000 per year.

Label: CONTRADICT
Article titile: Heat related deaths could rise from 2000 to 12 000 a year by the 2080s, health agency says
Abstract: ['More deaths of elderly people relating to hot weather, a longer hay fever season, and more with asthma are just some the public health implications climate change predicted for United Kingdom by detailed report published today Health Protection Agency.1\n\nThere will also be flooding, ultraviolet light, greater potential exotic, disease carrying mosquitoes invade UK, says HPA’s report, unveiled at its annual conference in Warwick.', 'Temperatures UK have been rising around 0.25°C decade since 1960s, while amount summer rainfall has decreased that winter increased.', 'It is mean average temperature rise another 2°C 5°C 2080 heat waves become frequent.', 'Southern-central eastern England expected see largest temperatures rises, rises being most